In [47]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from tqdm import tqdm_notebook
import pandas as pd
import time


In [48]:
# 브라우저 꺼짐 방지 옵션
chrome_options = Options()
chrome_options.headless=True
chrome_options.add_experimental_option("detach", True)


In [49]:
def scroll_down(driver):
    last_element = None  
    while True:
        url_raw = driver.find_elements(By.CSS_SELECTOR, '.JobCard_JobCard__Tb7pI a') 
        if not url_raw: 
            break
        if last_element == url_raw[-1]:  
            return url_raw
        last_element = url_raw[-1]  
        ActionChains(driver).move_to_element(last_element).perform()
        time.sleep(1)  

In [50]:
driver = webdriver.Chrome(options=chrome_options)
driver.set_window_size(1900, 1000)
# 웹페이지가 로드될 때까지 2초를 대기
driver.implicitly_wait(time_to_wait=2)  

# 원티드드 사이트로 이동합니다.
driver.get(url='https://www.wanted.co.kr/wdlist?country=kr&job_sort=job.latest_order&years=0&locations=all')
#직군 선택 
driver.find_element(By.XPATH, '//*[@id="__next"]/div[3]/div[1]/section/div[1]/section/div/button/span/span[2]').click()
driver.find_element(By.XPATH, '//*[@id="__next"]/div[3]/div[1]/section/div[1]/section/div/div/div[2]/div[2]/ul[1]/button[2]').click()
develope_oc = WebDriverWait(driver, 3).until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, "SelectItem_SelectItem__IcYKH"))
)
Occupation_list={'종료':0}
num=2
for button in develope_oc:
    try:
        occupation = WebDriverWait(button, 3).until(
            EC.presence_of_element_located((By.CLASS_NAME, "wds-1c3y3mo"))
        )
        Occupation_list[occupation.text]=num
        num+=1
    except:
        pass
print(Occupation_list)
time.sleep(1)
while True:
    input_occupation=input('직군선택: ')
    if input_occupation == '종료':
        break
    occupation=Occupation_list[input_occupation]
    driver.find_element(By.XPATH, f'//*[@id="__next"]/div[3]/div[1]/section/div[1]/section/div/div/div[2]/div[2]/ul[2]/button[{occupation}]/span').click()
driver.find_element(By.XPATH, '//*[@id="__next"]/div[3]/div[1]/section/div[1]/section/div/div/div[2]/div[3]/button[2]/span[2]').click()
time.sleep(1)
scroll_down(driver)
url_raw=scroll_down(driver)
url_list=[]
for url in url_raw:
    try:
        link = url.get_attribute("href")  
        url_list.append(link)
    except:
        pass
print(len(url_list))



{'종료': 0, '소프트웨어 엔지니어': 2, '서버 개발자': 3, '프론트엔드 개발자': 4, '웹 개발자': 5, '자바 개발자': 6, '파이썬 개발자': 7, '머신러닝 엔지니어': 8, 'C,C++ 개발자': 9, '데이터 엔지니어': 10, 'DevOps / 시스템 관리자': 11, 'Node.js 개발자': 12, '안드로이드 개발자': 13, '임베디드 개발자': 14, '시스템,네트워크 관리자': 15, 'QA,테스트 엔지니어': 16, '개발 매니저': 17, '보안 엔지니어': 18, 'iOS 개발자': 19, '데이터 사이언티스트': 20, '기술지원': 21, '하드웨어 엔지니어': 22, '프로덕트 매니저': 23, '빅데이터 엔지니어': 24, '웹 퍼블리셔': 25, '.NET 개발자': 26, '크로스플랫폼 앱 개발자': 27, 'PHP 개발자': 28, '블록체인 플랫폼 엔지니어': 29, '영상,음성 엔지니어': 30, 'DBA': 31, 'ERP전문가': 32, '그래픽스 엔지니어': 33, 'CTO,Chief Technology Officer': 34, '루비온레일즈 개발자': 35, 'BI 엔지니어': 36, 'VR 엔지니어': 37, 'CIO,Chief Information Officer': 38, 'RPA 엔지니어': 39, '테크니컬 라이터': 40}
58


In [54]:
driver = webdriver.Chrome(options=chrome_options)
driver.set_window_size(1900, 1000)
# 웹페이지가 로드될 때까지 2초를 대기
driver.implicitly_wait(time_to_wait=2)
data_dict={}

for n in tqdm_notebook(range(len(url_list)),desc="진행률"):
    data_info={}
    Main_task, qualifications, Preferred_Qualifications=None, None, None
    driver.get(url_list[n])
    driver.implicitly_wait(2)
    try:
        see_more=driver.find_elements(By.CLASS_NAME, 'Button_Button__root__m1NGq.Button_Button__outlined__0HnEd.Button_Button__outlinedAssistive__JKDyz.Button_Button__outlinedSizeLarge__A_H8o.Button_Button__fullWidth__zAnDP')
        for button in see_more:
            button.click()
    except:
        pass
    title = driver.find_element(By.CLASS_NAME, 'wds-jtr30u').text
    location = driver.find_element(By.CLASS_NAME, 'JobHeader_JobHeader__Tools__Company__Info__yT4OD.wds-rgovpd').text
    explanation_things=driver.find_elements(By.CSS_SELECTOR, "div.JobDescription_JobDescription__paragraph__Lhegj h3.wds-1y0suvb")
    explanation_detail=driver.find_elements(By.CSS_SELECTOR, "div.JobDescription_JobDescription__paragraph__Lhegj span.wds-wcfcu3")
    for i in range(len(explanation_things)):
        if explanation_things[i].text == '주요업무':
            Main_task=explanation_detail[i].text
        elif explanation_things[i].text == '자격요건':
            qualifications=explanation_detail[i].text
        elif explanation_things[i].text == '우대사항':
            Preferred_Qualifications=explanation_detail[i].text
    try:
        skils = driver.find_elements(By.CSS_SELECTOR, 'li.SkillTagItem_SkillTagItem__K3B3t span.wds-1m3gvmz')
        skils = [skil.text for skil in skils]
    except:
        skils=None
    
    data_info['title']=title
    data_info['location']=location
    data_info['Main_task']=Main_task
    data_info['qualifications']=qualifications
    data_info['Preferred_Qualifications']=Preferred_Qualifications
    data_info['skils']=skils
    data_info['url']=url_list[n]
    
    data_dict[n]=data_info
    time.sleep(0.5)
    print(n,title)
    
result_df=pd.DataFrame.from_dict(data_dict, 'index')
result_df.to_csv('wanted.csv', encoding='utf-8-sig', index=False)
print('저장완료',len(data_dict))

C:\Users\User\AppData\Local\Temp\ipykernel_21716\2797493281.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for n in tqdm_notebook(range(len(url_list)),desc="진행률"):


진행률:   0%|          | 0/58 [00:00<?, ?it/s]

0 딥러닝/머신러닝(AI) 개발자 [신입, 전문연구요원(보충역/전직) 가능
1 AI Engineer (병특 전문연구요원가능)
2 [이미지/비디오 생성형 AI] Research Engineer (전문연구요원)
3 Research Scientist
4 전문연구요원_AI 개발자 (병역 특례)
5 Operation / Digital Consultant
6 ML engineer/ AI researcher (전문연구요원 - 보충역, 전직자)
7 ML engineer/ AI researcher (Junior)
8 딥러닝 연구원 / 엔지니어 (전문연구요원 가능)
9 [AI연구소] AI연구원 - Computer Vision AI
10 신약개발 특허분석 인공지능 연구원
11 Computer Vision AI Engineer_Junior
12 Prompt Engineer
13 인공지능 음성 합성(TTS) 전문가
14 Physical AI Research Scientist (강화학습, 시뮬레이션)
15 [인턴] AI Team
16 NPU RTL 설계 엔지니어 (전문연구요원/신입)
17 AI Engineer(LLM)
18 [전문연구요원] Computer Vision Engineer
19 연구소 회로설계 및 HW 개발
20 AI 엔지니어(NLP/LLM)
21 인공지능(AI) 개발자
22 SW개발 AI 개발자
23 AI 엔지니어 (Stable diffusion / VITON)
24 딥러닝 엔지니어
25 머신러닝 엔지니어
26 딥러닝/머신러닝 분야 연구원 (인공지능 개발자)
27 CAD Engineer - 전문연구요원 가능
28 AI엔지니어
29 AI 최적화 Researcher
30 (전문연구요원 가능) ML Engineer/Researcher
31 (전문연구요원 가능) AI 컴파일러 엔지니어
32 AI engineer (prompt engineer)
33 LLM Researcher
34 Document AI Researcher
35 Generative AI 및 인공지능 유지 관리 연구


In [52]:
result_df.head()

,title,location,Main_task,qualifications,Preferred_Qualifications,skils,url
0,"딥러닝/머신러닝(AI) 개발자 [신입, 전문연구요원(보충역/전직) 가능",경기 성남시,• 생체 신호 기반 딥러닝/머신러닝 모델 개발\n• 내부 토론식 세미나를 통해 문제...,"* 지원 시 포트폴리오 제출 필수 (연구 자료, 논문 자료 등)\n• 딥러닝/머신러...","• 디지털 신호처리 알고리즘 개발 경험자\n• 통계학과, 응용수학과, 산업공학, 전...","[Git, Android, iOS, MySQL, Pytorch, React, Ten...",https://www.wanted.co.kr/wd/188191
1,AI Engineer (병특 전문연구요원가능),서울 종로구,[기술스택]\n-Language - Python\n-Framework - Pytor...,컴퓨터과학/데이터사이언스 전공자 또는 AI/ML 관련 전문 지식 보유자\n* 신입/...,-AI / ML 조직 리딩 경험\n-DevOps 엔지니어링 경험\n-AI / ML ...,"[Github, Pytorch, Tensorflow, Python, AWS, Doc...",https://www.wanted.co.kr/wd/255120
2,[이미지/비디오 생성형 AI] Research Engineer (전문연구요원),서울 강남구,• 비디오 생성 모델 개발 (video generation / frame inter...,"• 컴퓨터 공학, 전산학 또는 관련 분야 석사 이상 학위 소지자\n• AI 및 머신...",• 이미지/비디오 처리 관련 연구 경험\n• 대규모 데이터셋 처리 및 분석 경험\n...,[],https://www.wanted.co.kr/wd/263463
3,Research Scientist,서울 강남구,SwiftMR 제품 코드 중 모델 알고리즘 파트 코드 관리\nSwiftMR Rese...,"업무 맥락 파악 및 소통 능력 (예외 상황 발생 시 빠른 대처 및 투명한 소통, 담...",Python 코드 관리 경험\n수학과 친하신 분\n의료 AI분야에 관심이 있으신 분,[Python],https://www.wanted.co.kr/wd/258011
4,전문연구요원_AI 개발자 (병역 특례),경기 성남시,• 크라우드 펀딩 플랫폼을 위한 다양한 서비스에 적용되는 AI모델 설계\n• 서비스...,• 공학 석사 이상의 학위를 소지하고 계신 분\n• ai 를 활용한 실험 논문을 소...,• AWS 의 Serverless (람다) 혹은 k8s 환경에서의 서비스 오픈 경험...,"[Python, SQL]",https://www.wanted.co.kr/wd/263224
